In [3]:
import numpy as np
import pandas as pd
import scipy
import os
#---------------------Peleng fucntions ----------------------------------------------------------------------
def SetCSFunc(CSType):
    def GetDC(TX_R, TX_Az, TX_El):
        VDC = np.zeros(3)
        if CSType == 'AoE':
            VDC[0] = np.sin(TX_Az) * np.cos(TX_El)
            VDC[1] = np.sin(TX_El)
            VDC[2] = np.cos(TX_Az) * np.cos(TX_El)
        elif CSType == 'Z-AoE':
            VDC[0] = np.sin(TX_Az) * np.cos(TX_El) * (np.cos(TX_Az) * np.cos(TX_El)) ** (-1)
            VDC[1] = np.sin(TX_El) * (np.cos(TX_Az) * np.cos(TX_El)) ** (-1)
            VDC[2] = 1
        return VDC * TX_R
    return GetDC

def SetMRotate(n, Plane, angle):
    i = Plane[0]
    j = Plane[1]
    Res = np.identity(n)
    Res[i][j] = np.cos(angle)
    Res[i][j] = ((-1)**(i+j)) * np.sin(angle)
    Res[j][i] = - Res[i][j]
    Res[j][j] =  Res[i][i]
    return Res

def FSetOffset(TX_R, TX_Az, TX_El, VRX, IsPlaneWave, CSType):
    if IsPlaneWave == 1:
        return (TX_R, TX_Az, TX_El)
    else:
        FCS = SetCSFunc(CSType)
        VDC = FCS(1,TX_Az, TX_El)
        rho = mag(VDC)
        Dist = mag(TX_R * VDC - VRX)
        VDC[0] = VDC[0] * rho * TX_R / Dist
        VDC[1] = VDC[1] * rho * TX_R / Dist
        VDC[2] = cmag(np.array([rho, 1j * VDC[0], 1j*VDC[1]]))
        Az, El = GetAngles(CSType, VDC)
        return Dist, Az, El
    
def SetAPD(TX_R, TX_Az, TX_El, RX_Roll, VRX, F0, IsPlaneWave, CSType, FSetOffset):
    #CSType = lookup(CSType, VCSTypes, VCSRTypes) if IsPlaneWave == 1
    if IsPlaneWave == 1:
        FCS = SetCSFunc(CSType)
        VDC = FCS(1, TX_Az, TX_El)
        Buf = 0.5 * np.sqrt(2) *  SetMRotate(2,np.array([0,1]), -RX_Roll) @ (np.array([VDC[0],VDC[1]]))
        VDC[0], VDC[1], VDC[2] = Buf[0], Buf[1], 0
        Dist = mag(VRX) * np.sign(VRX) @ VDC
    else:
        VRX = np.matmul(SetMRotate(3, np.array([0,1]), RX_Roll) , VRX)
        Dist, _, _ = FSetOffset(TX_R, TX_Az, TX_El, VRX, IsPlaneWave, CSType)
    A = 1* FPlu(Dist + (1e3 if IsPlaneWave == 1 else 0), F0)
    P = GetAngle(((-1) ** (IsPlaneWave+1)) * Dist * GetK(F0))
    return A,P

def GetAngles(SType, VDC):
    R = cmag(VDC)
    U = np.real(VDC[0])
    V = np.real(VDC[1])
    W = np.real(VDC[2])
    if SType == 'AoE':
        try:
            Az = np.arcsin(U) if IsNaN(R) == 1 else np.arctan2(U, W)
            El = np.arcsin(V) if IsNaN(R) == 1 else np.arcsin(V / R)
        except:
            Az = None
            El = None
    if SType == 'Z-AoE':
        try:
            Az = np.arcsin(U) if IsNaN(R) == 1 else np.arctan2(U,W)
            El = np.arcsin(V) if IsNaN(R) == 1 else np.arcsin(V/R)
        except:
            Az = None
            El = None
    return np.real(Az), np.real(El)

def dB2VR(dB):
    return 10**(dB/20)

def lookup(z, A, B):
    i = 0
    while z != A[i]:
        i = i + 1
    return B[i]

def IsNaN(a):
    return 1 if a == None else 0

def cmag(V):
    return np.sqrt(np.matmul(V.transpose(), V))

def mag(V):
    return np.sqrt(np.matmul(V.transpose(), np.conjugate(V)))

def FPlu(D, F0):
    return (2 * D * GetK(F0)) ** (-1)

def GetK(F):
    return 2 * np.pi * (1e8 * 2.998 / F) ** (-1)

def SetSolver(Base,TX_R,RX_Z,delta,IsPlaneWave,IsRegularCS):
    #СК РЕГ
    IsRegularCS = np.max([IsRegularCS, IsPlaneWave])
    def GetF(DC,delta,Index,Sign):
        if IsPlaneWave == 0:
            Res = ((TX_R ** 2) * (Base ** 2) * (DC[1-Index] ** 2))  - delta ** 2 * (TX_R ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R * DC[2] + 0.5 * Base ** 2 + (-1) ** Sign * Base * TX_R * DC[Index] - 1/4 * delta ** 2)
        elif IsPlaneWave == 1:
            Res = delta + (-1) ** ((Index + Sign) % 2)  * Base * DC[1-Index]
        return Res

    def GetDeltaX(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = (-1) ** (Sign + 1) * Base * TX_R * delta ** 2
        else:
            Res = 0 # Заглушка
        return Res

    def GetDeltaY(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = 2 * Base ** 2 * TX_R ** 2 * DC[1 - Index]
        else:
            Res = (-1) ** ((Index + Sign) % 2) * Base # Заглушка
        return Res

    def GetDeltaW(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = 2 * RX_Z * TX_R * delta ** 2
        else:
            Res = 0 # Заглушка
        return Res

    # Заполняем Массива Градиента И Функции СК РЕГ
    def grad_RCS(DC):
        Res = np.empty((5,3), dtype = complex)
        Res[0][0] = GetDeltaX(DC, delta[3], 0, 0)
        Res[0][1] = GetDeltaY(DC, delta[3], 0, 0)
        Res[0][2] = GetDeltaW(DC, delta[3], 0, 0)

        Res[1][0] = GetDeltaY(DC, delta[0], 1, 0)
        Res[1][1] = GetDeltaX(DC, delta[0], 1, 0)
        Res[1][2] = GetDeltaW(DC, delta[0], 1, 0)

        Res[2][0] = GetDeltaX(DC, delta[1], 0, 1)
        Res[2][1] = GetDeltaY(DC, delta[1], 0, 1)
        Res[2][2] = GetDeltaW(DC, delta[1], 0, 1)

        Res[3][0] = GetDeltaY(DC, delta[2], 1, 1)
        Res[3][1] = GetDeltaX(DC, delta[2], 1, 1)
        Res[3][2] = GetDeltaW(DC, delta[2], 1, 1)

        Res[4][0] = 2 * DC[0]
        Res[4][1] = 2 * DC[1]
        Res[4][2] = 2 * DC[2]
        return Res

    def Func_RCS(DC):
        Res = np.empty((5), dtype = complex)
        Res[0] = GetF(DC, delta[3], 0,0)
        Res[1] = GetF(DC, delta[0], 1,0)
        Res[2] = GetF(DC, delta[1], 0, 1)
        Res[3] = GetF(DC, delta[2], 1, 1)
        Res[4] = cmag(DC) ** 2 -1
        return Res

    # СК НЕ РЕГ

    def GetF1(DC, delta, Index, Sign):
        Res = Base ** 2 * TX_R ** 2 * (DC[1-Index]) ** 2 - (DC[2]) ** 2 * delta ** 2 * (RX_Z ** 2 - 2 * RX_Z * TX_R + 0.5 * Base ** 2 - 0.25 * delta ** 2) - (TX_R ** 2 + (-1) ** Sign * Base * TX_R * DC[Index] * DC[2]) * delta ** 2
        return Res

    def GetDeltaX1(DC, delta, Index, Sign):
        Res = 2 * DC[Index] * delta ** 2 * (0.5 * Base ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R - 1/4 * delta ** 2) - (-1) ** Sign * Base * TX_R * DC[2] * delta ** 2
        return Res

    def GetDeltaY1(DC, delta, Index, Sign):
        Res = 2 * DC[1 - Index] * delta ** 2 * (0.5 * Base ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R - 1/4 * delta ** 2) + 2 * Base ** 2 * TX_R ** 2 * DC[1 - Index]
        return Res

    def GetDeltaW1(DC, delta, Index, Sign):
        Res = (-1) ** (Sign + 1) * Base * TX_R * DC[Index] * delta ** 2
        return Res

    def Func_ZCS(DC):
        Res = np.empty((5), dtype = complex)
        Res[0] = GetF1(DC, delta[3], 0,0)
        Res[1] = GetF1(DC, delta[0], 1,0)
        Res[2] = GetF1(DC, delta[1], 0, 1)
        Res[3] = GetF1(DC, delta[2], 1, 1)
        Res[4] = cmag(DC) -1
        return Res


    def grad_ZCS(DC):
        Res = np.empty((5,3), dtype = complex)

        Res[0][0] = GetDeltaX1(DC, delta[3], 0, 0)
        Res[0][1] = GetDeltaY1(DC, delta[3], 0, 0)
        Res[0][2] = GetDeltaW1(DC, delta[3], 0, 0)

        Res[1][0] = GetDeltaY1(DC, delta[0], 1, 0)
        Res[1][1] = GetDeltaX1(DC, delta[0], 1, 0)
        Res[1][2] = GetDeltaW1(DC, delta[0], 1, 0)

        Res[2][0] = GetDeltaX1(DC, delta[1], 0, 1)
        Res[2][1] = GetDeltaY1(DC, delta[1], 0, 1)
        Res[2][2] = GetDeltaW1(DC, delta[1], 0, 1)

        Res[3][0] = GetDeltaY1(DC, delta[2], 1, 1)
        Res[3][1] = GetDeltaX1(DC, delta[2], 1, 1)
        Res[3][2] = GetDeltaW1(DC, delta[2], 1, 1)

        Res[4][0] = 2 * DC[0]
        Res[4][1] = 2 * DC[1]
        Res[4][2] = 2 * DC[2]
        return Res
    return (Func_ZCS, grad_ZCS) if IsRegularCS == 0  else (Func_RCS, grad_RCS)

def DirFind(Vdx, Base, RX_Z, RX_Roll, TX_R, CSType, IsPlaneWave, N, eps):
    VCSTypes = ['Z-AoE', 'AoE', 'PS']
    VCSRTypes = ['AoE', 'AoE', 'PS']
    CSRType = lookup(CSType, VCSTypes, VCSRTypes)
    IsRegularCS = (CSType == CSRType)
    Func, FGrad = SetSolver(Base,TX_R,RX_Z,Vdx,IsPlaneWave,IsRegularCS)
    VDC0 = np.stack([0.5,0.5])
    VDC0 = np.append(VDC0, cmag(np.append(1j*VDC0,1)))
    WD_Max = 25
    WD = 0
    i = 0
    err = np.array([eps + 1])
    VDC = VDC0
    VDCN = np.empty(3,dtype=complex)
    while i<N if N > 0 else mag(err) > eps:
        J = FGrad(VDC)
        try:

            VDCN = VDC - np.matmul(np.matmul(np.linalg.matrix_power(np.matmul(J.transpose(),J),-1), J.transpose()) ,Func(VDC))
            #VDCN = VDC - np.matmul(J.transpose(), J) ** (-1), J.transpose(), Func(VDC)
        except:
            WD = WD + 1
            i = N if WD > WD_Max else 0
            VDCN[2] = cmag(np.append(1j*VDCN, 1)) if WD <= WD_Max else None
            VDCN[0] = np.random.rand(1)  if WD<=WD_Max else None
            VDCN[1] = np.random.rand(1)  if WD<=WD_Max else None
        err = VDCN - VDC
        VDC = VDCN
        i = i + 1
    VDC[0] = VDC[0] * (1 if IsPlaneWave == 1 else np.sign(Vdx[0]))
    VDC[1] = VDC[1] * (1 if IsPlaneWave == 1 else np.sign(Vdx[1]))
    VDC = np.matmul(SetMRotate(3, np.stack([0,1]), RX_Roll) , VDC)
    if IsPlaneWave == 1:
        VDC[2] = cmag(np.stack([1, 1j*VDC[0], 1j*VDC[1]],1j*VDC[2]))
    VDC[2] = cmag(np.stack([1, 1j*VDC[0], 1j*VDC[1]]))
    return GetAngles(CSType, VDC)
def SetMatrixRX(Base):
    MRX = np.zeros((4,2))
    MRX[0] = np.stack([-1,-1])
    MRX[1] = np.stack([1,-1])
    MRX[2] = np.stack([1,1])
    MRX[3] = np.stack([-1,1])
    return Base * MRX * 0.5
#-------------------------------------------------------------------------------------------------

def PR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 10 * np.log10(val)

def VR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 2 * PR2dB(val)

def GetAngle(alpha):
    return np.arctan2(np.sin(alpha), np.cos(alpha))

def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])

def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F

def read_data(file_dict : dict):
    absolute_path = os.path.join(file_dict['CD'],file_dict['file_name'])
    names = [['TimeStamp'],['Freq'],['Azimuth'],['Tilt']]
    element_MA = [['A{}_MAG'.format(i),'A{}_ARG'.format(i)] for i in range(4)]
    names.extend(element_MA)
    names = [item for element in names for item in element]
    data_frame = pd.read_csv(absolute_path,sep=';',comment = '#',header = None,names=names)
    return data_frame

def filter_data(df,freq,az,tilt,num_of_element,iloc):
    locls = locals()
    errors = 0
    for e in ['df', 'iloc', 'num_of_element']:
        locls.pop(e)
    for key in locls.keys():
        if locls[key] not in df.values:
            print(f'{key} = {locls[key]} not in DataFrame')
            errors = 1
    if errors == 1:
        return 0
    return df[((df.Freq == freq) & (df.Azimuth == az) & (df.Tilt == tilt))][['A{}_MAG'.format(num_of_element),'A{}_ARG'.format(num_of_element)]].iloc[iloc].to_numpy()

def convert_data(src,src_format, dst_format):
    if src_format == dst_format:
        return src
    else:
        src_mag_format, src_arg_format = src_format.split('|')
        dst_mag_format, dst_arg_format = dst_format.split('|')
        if src_mag_format != dst_mag_format:
            if src_mag_format == 'dB' and dst_mag_format == 'VR':
                src[0] = 10**(0.1*src[0])
            else:
                src[0] = PR2dB(src[0])
        if src_arg_format != dst_arg_format:
            if src_arg_format == 'deg' and dst_arg_format == 'rad':
                src[1] = np.deg2rad(src[1])
            else:
                src[1] = np.rad2deg(src[1])
        return src
    
def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])

def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F

def target_fuction(delta_rx,ampl_rx, VSRC, VINC,ERROR_CONST):
    VINC = VINC.reshape((4,2))
    VSRC_reversed = np.roll(VSRC,-1,axis=0)
    VINC_reversed = np.roll(VINC,-1,axis=0)
    F = np.zeros(8)
    for i in range(4):
        F[i] = np.angle(VSRC_reversed[i,0] + VINC_reversed[i,0] + 1j *(VSRC_reversed[i,1] + VINC_reversed[i,1])) -\
            np.angle(VSRC[i,0] + VINC[i,0] + 1j *(VSRC[i,1] + VINC[i,1])) - delta_rx[i] + ERROR_CONST[i]
    for i in range(4,8):
        F[i] = (VSRC[i-4,0] + VINC[i-4,0]) ** 2 + (VSRC[i-4,1] + VINC[i-4,1]) ** 2 - ampl_rx[i-4] ** 2
    return F


In [297]:
SUnits = {'mm' : 1e-3,'cm' : 1e-2,'MHz' : 1e6,'GHz' : 1e9}
Base = 7*SUnits['cm']
MRX = SetMatrixRX(Base)
NUM_OF_ELEMENTS = 4
AZ = 6
TILT = 0
FREQ = 3.5e9
TX_R = 20
RX_ROLL = 0
IS_PLANEWAVE = 1
df = read_data(dict(CD = os.getcwd(), file_name = 'Meas_27.05.23.txt'))
VSP0 = [convert_data(filter_data(df,freq = FREQ,az = 0,tilt = 0,num_of_element=i,iloc = 0),'dB|deg','VR|rad') for i in [3,2,1,0]]
VSRX = [convert_data(filter_data(df,freq = FREQ,az = AZ,tilt = TILT,num_of_element=i,iloc = 0),'dB|deg','VR|rad') for i in [3,2,1,0]]
phase_rx = np.zeros(NUM_OF_ELEMENTS)
ampl_rx = np.zeros(NUM_OF_ELEMENTS)
phase_p0 = np.zeros(NUM_OF_ELEMENTS)
for i in range(len(phase_rx)):
    phase_rx[i] = VSRX[i][1]
    phase_p0[i] = VSP0[i][1]
    ampl_rx[i] = VSRX[i][0]
VSRX = np.array(VSRX)
delta_phase = GetAngle(np.roll(phase_rx,-1) - phase_rx)
error_const = GetAngle(np.roll(phase_p0,-1) - phase_p0)
VSRC = np.zeros((NUM_OF_ELEMENTS,2))
phase_src = np.zeros(NUM_OF_ELEMENTS)
for i in range(NUM_OF_ELEMENTS):
    A,P = SetAPD(10, np.deg2rad(AZ), -np.deg2rad(TILT), RX_ROLL, np.stack([MRX[i][0],MRX[i][1],0]), FREQ, IS_PLANEWAVE, 'Z-AoE', FSetOffset)
    phase_src[i] = P
    complex_value = A * np.exp(1j*P)
    VSRC[i,:] = [np.real(complex_value) , np.imag(complex_value)] 
delta_phase_src = GetAngle(np.roll(phase_src,-1) - phase_src)
VX0 = np.zeros(NUM_OF_ELEMENTS * 2)
print(VSRX)
VX0 = VSRX.reshape((1,NUM_OF_ELEMENTS * 2))
VX0 = VSRC.reshape((1,NUM_OF_ELEMENTS * 2))
func = lambda VINC: target_fuction(delta_phase,ampl_rx, VSRC, VINC,error_const)
result = scipy.optimize.fsolve(func,VX0,maxfev = 10000,xtol=1e-10)
result = result.reshape(NUM_OF_ELEMENTS,2)
VSC = VSRX - result
index = 1
phase_compensated = np.array([(np.angle(VSC[index,0] + 1j*VSC[index,1])) for index in range(NUM_OF_ELEMENTS)])
delta_phase_comp = (np.roll(phase_compensated,-1) - phase_compensated)
print(np.rad2deg(delta_phase))
print(np.rad2deg(delta_phase_src))
print(np.rad2deg(delta_phase_comp))
print(f'compensated phase = {np.rad2deg(np.angle(VSC[index,0] + 1j*VSC[index,1]))}, received phase = {np.rad2deg(phase_rx[index])}, P = {np.rad2deg(P)}')


[[ 3.12903917e-04 -1.49936878e+00]
 [ 3.42364243e-04 -2.20180905e+00]
 [ 2.48173268e-04 -1.89206659e+00]
 [ 1.49375097e-04 -1.85253950e+00]]
[-40.246863  17.746936   2.264735  20.235192]
[ 30.92125934   0.         -30.92125934   0.        ]
[ 0.00051937  0.00087159 -0.00123375 -0.00015721]
compensated phase = -89.99922201428257, received phase = -126.154366, P = -15.460629670501442


In [298]:
VSRC + result 

array([[ 3.12705842e-04,  1.11318310e-05],
       [ 3.19038579e-04, -1.24208131e-04],
       [ 2.00273739e-04, -1.46561935e-04],
       [ 1.42499657e-04,  4.47969568e-05]])

In [299]:
VSRX

array([[ 3.12903917e-04, -1.49936878e+00],
       [ 3.42364243e-04, -2.20180905e+00],
       [ 2.48173268e-04, -1.89206659e+00],
       [ 1.49375097e-04, -1.85253950e+00]])

In [293]:
VSC

array([[ 6.76781957e-06, -1.49938173e+00],
       [ 2.98953599e-05, -2.20168303e+00],
       [ 5.44692257e-05, -1.89191821e+00],
       [ 1.34451845e-05, -1.85258612e+00]])

In [294]:
VSRX

array([[ 3.12903917e-04, -1.49936878e+00],
       [ 3.42364243e-04, -2.20180905e+00],
       [ 2.48173268e-04, -1.89206659e+00],
       [ 1.49375097e-04, -1.85253950e+00]])

In [237]:
# DO SOME TEST 
error_const = np.zeros(4)
VSSRC = np.array([[1,0.5],[0.2,0.75],[1,0.5],[1,0.5]])
VSINC = np.array([[1,2],[0.1,2.5],[2,6],[7,3]])
phase_src= np.array([(np.angle(VSSRC[index,0] + 1j*VSSRC[index,1])) for index in range(NUM_OF_ELEMENTS)])
delta_phase_src = GetAngle(np.roll(phase_src,-1) - phase_src)
VSSRC_reversed = np.roll(VSSRC,-1,axis = 0)
VSINC_reversed = np.roll(VSINC,-1,axis = 0)
VSRX = VSSRC + VSINC
ampl_rx = [np.abs(VSRX[i,0] + 1j * VSRX[i,1]) for i in range(4)]
VSRX_reversed = np.roll(VSRX,-1,axis = 0)
delta_phase = [np.arctan2(VSRX_reversed[i,1],VSRX_reversed[i,0])-np.arctan2(VSRX[i,1],VSRX[i,0]) for i in range(4)]
#target_fuction(delta_phase,ampl_rx, VSSRC, VSINC,error_const)
func = lambda VINC: target_fuction(delta_phase,ampl_rx, VSSRC, VINC,error_const)
result = scipy.optimize.root(func,VSRX,method = 'hybr')
result = result.x
result = result.reshape(4,2)
# compl_res = [(result[i,0]+1j*result[i,1])*np.exp(-1j*np.pi) for i in range(NUM_OF_ELEMENTS)]
# result_real, result_compl = (np.array([np.real(compl_res[i]) for i in range(NUM_OF_ELEMENTS)]), np.array([np.imag(compl_res[i]) for i in range(NUM_OF_ELEMENTS)]))
# result = np.column_stack((result_real,result_compl))
print(f'result = {result}')
VSC = VSRX - result
print(f'VSC = {VSC}')
print(f'VSSRC = {VSSRC}')
print(f'VSRX = {VSRX}')
phase_compensated = np.array([(np.angle(VSC[index,0] + 1j*VSC[index,1])) for index in range(NUM_OF_ELEMENTS)])
delta_phase_comp = GetAngle(np.roll(phase_compensated,-1) - phase_compensated)
print(f'delta comp  = {np.rad2deg(delta_phase_comp)}')
print(f'delta rx = {np.rad2deg(delta_phase)}')
print(f'delta src = {np.rad2deg(delta_phase_src)}')
print(f'phase com = {np.rad2deg(phase_compensated)}')

result = [[ 0.57810547  2.28560284]
 [-0.41934108  2.50643816]
 [ 0.93092987  6.39358469]
 [ 6.34350704  4.22471209]]
VSC = [[ 1.42189453  0.21439716]
 [ 0.71934108  0.74356184]
 [ 2.06907013  0.10641531]
 [ 1.65649296 -0.72471209]]
VSSRC = [[1.   0.5 ]
 [0.2  0.75]
 [1.   0.5 ]
 [1.   0.5 ]]
VSRX = [[2.   2.5 ]
 [0.3  3.25]
 [3.   6.5 ]
 [8.   3.5 ]]
delta comp  = [ 37.37391725 -43.00432723 -26.57348575  32.20389573]
delta rx = [ 33.3859123  -19.50124461 -41.5954817   27.71081402]
delta src = [ 48.50353164 -48.50353164   0.           0.        ]
phase com = [  8.57462183  45.94853908   2.94421185 -23.6292739 ]
[  10.80886607 -118.07291005  -53.13853693    5.63884455]


In [218]:
VSRX - VSINC == VSSRC

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [212]:
func(result)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -3.55271368e-15,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [154]:
func(VSINC)

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [159]:
VSINC + VSSRC

array([[27.,  1.],
       [29.,  1.],
       [27.,  1.],
       [28.,  1.]])

In [168]:
VSRX 

array([[27.,  2.],
       [29.,  2.],
       [27.,  2.],
       [28.,  2.]])

In [184]:
result

array([[1.14233728, 3.93400395],
       [1.14233728, 3.93400395],
       [1.14233728, 3.93400395],
       [1.14233728, 3.93400395]])

array([[ 39.1986504 , -20.89531523],
       [ 37.84039929, -20.12840039],
       [ 35.12389705, -18.59457071],
       [ 36.48214817, -19.36148555]])

In [62]:
print(np.rad2deg(np.angle(result[3,0] + 1j*result[3,1])))
print(np.rad2deg(np.angle(VSINC[1,0] + 1j*VSINC[1,1])))

74.49370478269131
1.4320961841646465
